In [125]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [126]:
%autoreload

import pandas as pd
import process as pr
import os

In [127]:
current_dir = ''

In [128]:
dataset_base_path = os.path.join(current_dir, 'dataset', 'dataset')

In [129]:
def get_all_user_data(data_path):
    # create for each user entry in list to insert them all in once
    all_users = os.listdir(data_path)
    # transform it in tuple with (id, has_labels) set has_labels to False for all
    all_users = [(u, False) for u in all_users]
    with open(os.path.join(dataset_base_path, 'labeled_ids.txt')) as f:
        for line in f.readlines():
            # update all entries with idx in file to has_labels = True
            all_users[int(line.strip())] = (all_users[int(line.strip())][0], True)
    
    return all_users



In [130]:
def upload_single_activity(data):
    query = "INSERT INTO Activity (id, user_id, transportation_mode, start_date_time, end_date_time) VALUES" + data
    cursor.execute(query)

In [131]:
def upload_trackpoints_batch(data):
    cursor.executemany("""INSERT INTO TrackPoint (activity_id, lat, long, altitude, date_days, date_time)
    VALUES (%s, %s, %s, %s, %s, %s)""", data)

In [132]:
def get_labels_as_df(file_path):
    temp = pd.read_csv(file_path, sep='\t', parse_dates=['Start Time','End Time'], date_format='yyyy/mm/dd hh:mm:ss')
    temp['Start Time'] = pd.to_datetime(temp['Start Time'])
    temp['End Time'] = pd.to_datetime(temp['End Time'])
    return temp

In [133]:
test_df = get_labels_as_df('dataset/dataset/Data/010/labels.txt')
test_df.head()

,Start Time,End Time,Transportation Mode
0,2007-06-26 11:32:29,2007-06-26 11:40:29,bus
1,2008-03-28 14:52:54,2008-03-28 15:59:59,train
2,2008-03-28 16:00:00,2008-03-28 22:02:00,train
3,2008-03-29 01:27:50,2008-03-29 15:59:59,train
4,2008-03-29 16:00:00,2008-03-30 15:59:59,train


In [134]:
points = pr.get_trackpoints_as_df('dataset/dataset/Data/010/Trajectory/20070804033032.plt')
list(points.itertuples(index=False, name=None))

[(Timestamp('2007-08-04 03:30:32'),
  39.921712,
  116.472343,
  13,
  39298.1462037037,
  '03:30:32'),
 (Timestamp('2007-08-04 03:30:33'),
  39.921705,
  116.472343,
  13,
  39298.1462152778,
  '03:30:33'),
 (Timestamp('2007-08-04 03:30:34'),
  39.921695,
  116.472345,
  13,
  39298.1462268519,
  '03:30:34'),
 (Timestamp('2007-08-04 03:30:35'),
  39.921683,
  116.472342,
  13,
  39298.1462384259,
  '03:30:35'),
 (Timestamp('2007-08-04 03:30:36'),
  39.921672,
  116.472342,
  13,
  39298.14625,
  '03:30:36'),
 (Timestamp('2007-08-04 03:30:38'),
  39.921583,
  116.472315,
  13,
  39298.1462731481,
  '03:30:38'),
 (Timestamp('2007-08-04 03:30:39'),
  39.921572,
  116.472315,
  13,
  39298.1462847222,
  '03:30:39'),
 (Timestamp('2007-08-04 03:30:40'),
  39.92156,
  116.47229,
  13,
  39298.1462962963,
  '03:30:40'),
 (Timestamp('2007-08-04 03:30:41'),
  39.921565,
  116.47229,
  13,
  39298.1463078704,
  '03:30:41'),
 (Timestamp('2007-08-04 03:30:42'),
  39.92157,
  116.472288,
  13,
  39

check for matching between labels and df of activity (check if start time of first row is equal to start time of label and 
end time of last row equals end time of labels)

create list of tuples with (activity id, min_time, max_time) and search for matchings. use activity id to reference and add attribute

In [137]:
# for user in os.listdir('dataset/dataset/Data'):
current_activity_id = 0
user = '010'
base_path = f'dataset/dataset/Data/{user}'
labels_exists = os.path.exists(f'{base_path}/labels.txt')
if labels_exists:
    df_labels = get_labels_as_df(f'{base_path}/labels.txt')
for activity in os.listdir(f'{base_path}/Trajectory'):
# activity = '20070804033032.plt'

    path_activity = f'{base_path}/Trajectory/{activity}'
    activity_df = pr.get_trackpoints_as_df(path_activity)
    if activity_df.shape[0] <= 2500:
        start_time = activity_df.head(1).values[0][0]
        end_time = activity_df.tail(1).values[0][0]
        transportation_label = ''
        if labels_exists:
            matching_row = df_labels.loc[(df_labels['Start Time'] == start_time) & ((df_labels['End Time']) == end_time)]
            if matching_row.shape[0] > 0:
                transportation_label = matching_row.values[0][2]
        
        upload_single_activity((current_activity_id, int(user), transportation_label, start_time, end_time))

    activity_df.drop(columns=['Date'], inplace=True)
    activity_df['activity_id'] = current_activity_id
    activity_df = activity_df.reindex(columns=['activity_id', 'Latitude', 'Longitude', 'Altitude', 'date_days', 'date_time'])
    upload_trackpoints_batch(list(activity_df.itertuples(index=False, name=None)))
    current_activity_id += 1
        

      activity_id   Latitude   Longitude  Altitude     date_days date_time
0               0  39.921712  116.472343        13  39298.146204  03:30:32
1               0  39.921705  116.472343        13  39298.146215  03:30:33
2               0  39.921695  116.472345        13  39298.146227  03:30:34
3               0  39.921683  116.472342        13  39298.146238  03:30:35
4               0  39.921672  116.472342        13  39298.146250  03:30:36
...           ...        ...         ...       ...           ...       ...
1111            0  39.902912  116.421455       180  39298.176759  04:14:32
1112            0  39.902908  116.421432       180  39298.176771  04:14:33
1113            0  39.902903  116.421413       180  39298.176794  04:14:35
1114            0  39.902892  116.421330       180  39298.176910  04:14:45
1115            0  39.902885  116.421300       180  39298.176921  04:14:46

[1116 rows x 6 columns]
     activity_id   Latitude   Longitude  Altitude     date_days date_time
0

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.